In [14]:
import pandas as pd
import sys

import plotly.graph_objects as go
import numpy as np
import math

In [15]:
# Go into above directory and download function
sys.path.append('../') 
import sheet_func

sheet_id = 867359064
name = "par_algo"

df = sheet_func.sheet_to_df(sheet_id, name)
df

,,Family Name,Looked at?,Variation,Algo ID,Algorithm Description,Final Call,Exact Problem Statement?,Exact?,Time Complexity (Average),...,n = 10^6 value,n = 10^6 scale,n = 10^6 Rate,n = 10^9 value,n = 10^9 scale,n = 10^9 Rate,Starting Complexity,Remarks,Papers for ratio evaluations,Domains
0,1.0,Sorting,1.000,Comparison Sorting,17.0,NaN,NaN,1.0,1.0,O(log²n),...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Combinatorics
1,1.0,Sorting,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Combinatorics
2,1.0,Sorting,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Combinatorics
3,1.0,Sorting,0.000,NaN,NaN,Bitonic merge-exchange,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Combinatorics
4,1.0,Sorting,0.001,NaN,NaN,"network sorting algo, perfect shuffle",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,NaN,All-Pairs Shortest Paths (APSP),0.000,NaN,NaN,Warshall-Floyd method,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,NaN,All-Pairs Shortest Paths (APSP),1.000,NaN,NaN,Repeated plus-min method,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,NaN,All-Pairs Shortest Paths (APSP),1.000,NaN,NaN,Repeated plus-min method,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,NaN,Matrix Product,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 75 columns):
 #   Column                                                                     Non-Null Count  Dtype  
---  ------                                                                     --------------  -----  
 0                                                                              143 non-null    float64
 1   Family Name                                                                151 non-null    object 
 2   Looked at?                                                                 151 non-null    float64
 3   Variation                                                                  69 non-null     object 
 4   Algo ID                                                                    41 non-null     float64
 5   Algorithm Description                                                      43 non-null     object 
 6   Final Call                                                

In [17]:
df.columns

Index([' ', 'Family Name', 'Looked at?', 'Variation', 'Algo ID',
       'Algorithm Description', 'Final Call', 'Exact Problem Statement?',
       'Exact?', 'Time Complexity (Average)', 'Average Case Distribution',
       'Reference?', 'Unnamed: 12', 'Algorithm Name', 'Year',
       'Paper/Reference Link', 'Constants', 'Derived?',
       'Paper Reference for Constants', 'Time Complexity Improvement?',
       'Transition Class', 'Time Complexity Class', 'Param: Time Class',
       'Time Complexity (Worst Only)', 'Parallel Algorithm Span',
       'Parallel Algorithm Span References', 'Parallel Algorithm Work',
       'Parallel Algorithm Work References',
       'Reference mentions work efficiency?', 'Parameter definitions',
       'Preferred Parameter', 'Time Complexity Reference',
       'Derived Time Complexity?', 'Computational Model', 'Unit of Space',
       'Space Complexity Class', 'Param: Space Class',
       'Space Complexity (Auxiliary)', 'Space Complexity Reference',
       'Der

In [18]:
# Remove unnecesary columns from the table to make the df more usable
columns_dropped = [' ','Reference?', 'Paper/Reference Link', 'Paper Reference for Constants',
        'Looked at?',   'Parallel Algorithm Span References', 
       'Parallel Algorithm Work References','Reference mentions work efficiency?',
       'Space Complexity Reference', 'Other References',
       'Algorithm family\nparameters', 'Description of Inputs',
       'Ratios of input parameter sizes', 'Space n = 1000 value',
       'Space n = 1000 rate', 'Space n = 10^6 value', 'Space n = 10^6 rate',
       'Space n = 10^9 value', 'Space n = 10^9 rate', 'n = 1000 value',
       'n = 1000 scale', 'n = 1000 Rate', 'n = 10^6 value', 'n = 10^6 scale',
       'n = 10^6 Rate', 'n = 10^9 value', 'n = 10^9 scale', 'n = 10^9 Rate',
       'Starting Complexity', 'Remarks', 'Papers for ratio evaluations',
       'Domains', 'Final Call']
df = df.drop(columns=columns_dropped, axis=1)

### What should we do with this data? How should we visualize it?
Here are some ideas:
- Start out by graphing any one particular algorithms complexity compared to others
- Sort out decrease in algorithmic complexity over time by family


1. Create separate DF's for each family name

In [19]:
# Identify all unique family names
unique_families = df['Family Name'].unique()

# Create a dictionary to store the grouped data frames
grouped_dfs = {}

# Add family names into a dictionary with the data. Sort by year.
for family in unique_families:
    grouped_df = df[df['Family Name'] == family].copy()
    grouped_dfs[family] = grouped_df.sort_values('Year')    # key:familyname val:data

In [20]:
# Remove rows with missing values
# grouped_dfs['Sorting'][''].dropna(axis=0, inplace=True)
# grouped_dfs['Sorting']['Year']

2. Create a function that will convert a given time complexity into a numerical ranking from 1 to n where n is the number of unique time complexities

In [21]:
complexity_rankings = {
        'O(1)': 1,
        'O(log²n)':2,
        'O(logn)': 3,
        'O(n)': 4,
        'O(nlogn)': 5,
        'O(n²)': 6,
        'O(2^n)': 7,
        'O(n!)': 8,
    }

def create_complexity_rank(df, complexity_column):
    # Map complexities to their ranks and handle NaN values
    df['complexity rank'] = df[complexity_column].map(complexity_rankings).fillna('Undefined')
    return df

In [22]:
# Run the 'Time Complexity (Average)' column through the function
grouped_dfs['Sorting'] = create_complexity_rank(grouped_dfs['Sorting'], 'Time Complexity (Average)')
# grouped_dfs['Sorting']

3. Plot a histogram of the complexity rank vs the year 

In [23]:
# Filter out rows with 'Undefined' complexity rank
filtered_sorting = grouped_dfs['Sorting'].loc[grouped_dfs['Sorting']['complexity rank'] != 'Undefined', ['Year', 'complexity rank']]

In [24]:
# Create a bar chart
fig = go.Figure(data=go.Bar(
    x=filtered_sorting['Year'],
    y=filtered_sorting['complexity rank'],
    marker=dict(
        color=filtered_sorting['complexity rank'],
        colorscale='agsunset'
        
    )
))

# Update the layout
fig.update_layout(
    title='Complexity Rank Distribution',
    xaxis_title='Year',
    yaxis_title='Complexity Rank'
)

# Show the figure
fig.show()

In [25]:
import plotly.graph_objects as go
import numpy as np
import math

# Input size
x = np.linspace(1, 1000, 100)

# Time complexities
y1 = np.ones_like(x)  # O(1)
y2 = np.log(x)  # O(n)
y3 = x  # O(log(n))
y4 = x * np.log(x)  # O(n*log(n))
y5 = x**2  # O(n^2)
y6 = 2**x  # O(2^n)
# y7 = [math.factorial(int(n)) for n in x]  # O(n!)

# Create traces
trace1 = go.Scatter(x=x, y=y1, mode='lines', name='O(1)')
trace2 = go.Scatter(x=x, y=y2, mode='lines', name='O(log(n))')
trace3 = go.Scatter(x=x, y=y3, mode='lines', name='O(n)')
trace4 = go.Scatter(x=x, y=y4, mode='lines', name='O(n*log(n))')
trace5 = go.Scatter(x=x, y=y5, mode='lines', name='O(n^2)')
trace6 = go.Scatter(x=x, y=y6, mode='lines', name='O(2^n)')
# trace7 = go.Scatter(x=x, y=y7, mode='lines', name='O(n!)')

# Layout
layout = go.Layout(
    title='Time Complexity Comparison',
    xaxis=dict(title='Input Size', range=[1, 100]),  # Set the x-axis range to [1, 1000]
    yaxis=dict(title='Time Complexity', range=[1, 100]),  # Set the y-axis range to [1, 1000]
)

# Create figure and add traces
fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5, trace6], layout=layout)

# Show the figure
fig.show()

In [26]:
# Input size
x = np.linspace(1, 1000, 100)

# Time complexities
y1 = np.ones_like(x)  # O(1)
y2 = x  # O(n)
y3 = np.log(x)  # O(log(n))
y4 = x * np.log(x)  # O(n*log(n))
y5 = x**2  # O(n^2)
y6 = 2**x  # O(2^n)
# y7 = [math.factorial(int(n)) for n in x]  # O(n!)

# Create traces
trace1 = go.Scatter(x=x, y=y1, mode='lines', name='O(1)', fill='tozeroy', fillcolor='rgba(0, 0, 255, 0.1)')
trace2 = go.Scatter(x=x, y=y2, mode='lines', name='O(n)', fill='tozeroy', fillcolor='rgba(255, 0, 0, 0.1)')
trace3 = go.Scatter(x=x, y=y3, mode='lines', name='O(log(n))', fill='tozeroy', fillcolor='rgba(0, 255, 0, 0.1)')
trace4 = go.Scatter(x=x, y=y4, mode='lines', name='O(n*log(n))', fill='tozeroy', fillcolor='rgba(255, 0, 255, 0.1)')
trace5 = go.Scatter(x=x, y=y5, mode='lines', name='O(n**2)', fill='tozeroy', fillcolor='rgba(255, 255, 0, 0.1)')
trace6 = go.Scatter(x=x, y=y6, mode='lines', name='O(2**n)', fill='tozeroy', fillcolor='rgba(0, 255, 255, 0.1)')
# trace7 = go.Scatter(x=x, y=y7, mode='lines', name='O(n!)')

# Layout
layout = go.Layout(
    title='Time Complexity Comparison',
    xaxis=dict(title='Input Size', range=[1, 100]),  # Set the x-axis range to [1, 1000]
    yaxis=dict(title='Time Complexity', range=[1, 100]),  # Set the y-axis range to [1, 1000]
)

# Create figure and add traces
base_graph = go.Figure(data=[trace1, trace2, trace3, trace4, trace5, trace6], layout=layout)

# Show the figure
base_graph.show()

In [27]:
def plot_complexity(complexities: np.array, x = np.linspace(1, 1000, 100), base_graph=go.Figure(base_graph)):

    """ Given an array of complexities and their family name as the label,
        display a time complexity comparison chart comparing the given complexities.
         
        Complexities take the form: [[np.log(x)*np.log(x), 'Family1'], 
                                     [np.log(x)**2, 'Family2']]   """
    
    
    for i, (complexity, label) in enumerate(complexities):
        trace = go.Scatter(x=x, y=complexity, mode='lines', name=f'{label}',  line=dict(width=10-2*i))
        base_graph.add_trace(trace)

    base_graph.show()
    return
    

print(plot_complexity([[np.log(np.log(x)), 'Family1'], 
                       [np.log(x)**2, 'Family2']]))

C:\Users\samin\AppData\Local\Temp\ipykernel_17956\4031251738.py:18: RuntimeWarning:

divide by zero encountered in log



None


In [28]:
# Input size
x = np.linspace(1, 1000, 100)

# Time complexities
y1 = np.ones_like(x)  # O(1)
y2 = x  # O(n)
y3 = np.log(x)  # O(log(n))
y4 = x * np.log(x)  # O(n*log(n))
y5 = x**2  # O(n^2)
y6 = 2**x  # O(2^n)

# Calculate midpoints
midpoints = [(y1 + y2) / 2, (y2 + y3) / 2, (y3 + y4) / 2, (y4 + y5) / 2, (y5 + y6) / 2]

# Create traces
trace1 = go.Scatter(x=x, y=y1, mode='lines', name='O(1)', fill='tozeroy')
trace2 = go.Scatter(x=x, y=y2, mode='lines', name='O(n)', fill='tozeroy')
trace3 = go.Scatter(x=x, y=y3, mode='lines', name='O(log(n))', fill='tozeroy')
trace4 = go.Scatter(x=x, y=y4, mode='lines', name='O(n*log(n))', fill='tozeroy')
trace5 = go.Scatter(x=x, y=y5, mode='lines', name='O(n^2)', fill='tozeroy')
trace6 = go.Scatter(x=x, y=y6, mode='lines', name='O(2^n)', fill='tozeroy')

# Create colors
c1 = 'rgba(0, 0, 255, 0.1)'
c2 = 'rgba(255, 0, 0, 0.1)'
c3 = 'rgba(0, 255, 0, 0.1)'
c4 = 'rgba(255, 0, 255, 0.1)'
c5 = 'rgba(255, 255, 0, 0.1)'
# Create boundary traces
boundary_trace1 = go.Scatter(x=x, y=midpoints[0], mode='lines', name='Boundary', fill='tozeroy', fillcolor=c1)
boundary_trace2 = go.Scatter(x=x, y=midpoints[1], mode='lines', name='Boundary', fill='tozeroy', fillcolor=c2)
boundary_trace3 = go.Scatter(x=x, y=midpoints[2], mode='lines', name='Boundary', fill='tozeroy', fillcolor=c3)
boundary_trace4 = go.Scatter(x=x, y=midpoints[3], mode='lines', name='Boundary', fill='tozeroy', fillcolor=c4)
boundary_trace5 = go.Scatter(x=x, y=midpoints[4], mode='lines', name='Boundary', fill='tozeroy', fillcolor=c5)

# Layout
layout = go.Layout(
    title='Time Complexity Comparison with Boundaries',
    xaxis=dict(title='Input Size', range=[1, 100]),
    yaxis=dict(title='Time Complexity', range=[1, 100]),
)

# Create figure and add traces
fig = go.Figure(data=[boundary_trace1, boundary_trace2, boundary_trace3, boundary_trace4,
                      trace1, trace2, trace3, trace4, trace5, trace6,],layout=layout)

# Show the figure
fig.show()